In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random, logging
from collections import deque
import tensorflow as tf
from tensorflow.keras import layers
from scipy.optimize import differential_evolution
import random
from tensorflow import keras
import gym
from matplotlib import pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

In [2]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [3]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

CONFIG = {
    "mutation_strategies": ['best1bin', 'rand1bin', 'rand2bin', 'currenttobest1bin', 'best1exp', 'rand1exp'],
    "crossover_strategies": ['bin', 'exp'],
    "cma_step_size": 0.3,
    "egt_max_size": 50,
    "dqn_learning_rate": 0.001,
    "max_generations": 100,
    "num_runs": 10,
    "batch_size": 32}

In [4]:
# EGT-based Memory Archive
class EGTMemory:
    def __init__(self, max_size=50):
        self.solutions = []
        self.fitness = []
        self.max_size = max_size

    def add_solution(self, solution, fitness):
        if len(self.solutions) < self.max_size:
            self.solutions.append(solution)
            self.fitness.append(fitness)
        else:
            worst_idx = np.argmax(self.fitness)
            if fitness < self.fitness[worst_idx]:
                self.solutions[worst_idx] = solution
                self.fitness[worst_idx] = fitness

    def extract_patterns(self):
        return np.mean(self.solutions, axis=0) if self.solutions else None

# DQN Agent for Adaptive Strategy Selection
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(self.state_size,)),
            tf.keras.layers.Dense(24, activation='relu'),
            tf.keras.layers.Dense(24, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def select_action(self, state):
        state = np.array(state).reshape(1, -1)
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def train(self, batch_size=32):
        if len(self.memory) < batch_size:
            return
        batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state in batch:
            target = reward + self.gamma * np.max(self.model.predict(np.array(next_state).reshape(1, -1), verbose=0))
            target_f = self.model.predict(np.array(state).reshape(1, -1), verbose=0)
            target_f[0][action] = target
            self.model.fit(np.array(state).reshape(1, -1), target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

class AdaptiveEA:
    def __init__(self, objective_function, bounds, generations=50, num_runs=10):
        self.bounds = bounds
        self.objective_function = objective_function
        self.generations = generations
        self.num_runs = num_runs
        self.agent = DQNAgent(state_size=1, action_size=len(CONFIG["mutation_strategies"]))
        self.egt_memory = EGTMemory()

    def evolve(self):
        all_runs_fitness_history = []
        for _ in range(self.num_runs):
            best_solution, best_fitness, fitness_history = self._evolve_single_run()
            all_runs_fitness_history.append(fitness_history)
        return best_solution, best_fitness, np.mean(all_runs_fitness_history), np.max(all_runs_fitness_history), np.std(all_runs_fitness_history), all_runs_fitness_history

    def _evolve_single_run(self):
        best_solution, best_fitness = None, float('inf')
        fitness_history = []
        strategies = CONFIG["mutation_strategies"]

        for generation in range(self.generations):
            strategy_idx = self.agent.select_action([generation])
            mutation_strategy = strategies[strategy_idx]

            result = differential_evolution(
                self.objective_function,
                bounds=self.bounds,
                strategy=mutation_strategy,
                recombination=0.7,
                popsize=20,
                tol=0.01,
                maxiter=1,
                disp=False)

            if result.fun < best_fitness:
                best_fitness = result.fun
                best_solution = result.x

            self.egt_memory.add_solution(result.x, result.fun)
            fitness_history.append(result.fun)

            reward = -result.fun
            next_state = [generation + 1]
            self.agent.memory.append(([generation], strategy_idx, reward, next_state))
            self.agent.train()
        return best_solution, best_fitness, fitness_history

In [5]:
def compute_metrics(fitness_history_obj):
    optimum, _, mean_fitness, worst_fitness, std_fitness, all_runs_fitness_history = fitness_history_obj.evolve()
    best_fitness_values = [np.min(run_fitness_history) for run_fitness_history in all_runs_fitness_history]
    worst_fitness_values = [np.max(run_fitness_history) for run_fitness_history in all_runs_fitness_history]

    return optimum, best_fitness_values, worst_fitness_values, mean_fitness, std_fitness

## **Rosenbrock**

In [ ]:
import time

def rosenbrock(x):
    return sum(100 * (x[i+1] - x[i]**2)**2 + (x[i] - 1)**2 for i in range(len(x)-1))

def evaluate_function(func, x):
    return func(x)

test_functions = {"Rosenbrock": rosenbrock}
bounds = [(-5, 5)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history = AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

In [ ]:
import pandas as pd
df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
df_results

,Function,Optimum,Best,Worst,Mean,Std,Execution Time (s)
0,Rosenbrock,"[1.0000000126756534, 0.9999999575401498, 0.999...",1.746331e-11,3.986579,0.76941,1.573315,8771.748088


## **Rastrigin**

In [ ]:
def rastrigin(x):
    return 10 * len(x) + sum(x_i**2 - 10 * np.cos(2 * np.pi * x_i) for x_i in x)

test_functions = {"Rastrigin": rastrigin}
bounds = [(-5.12, 5.12)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])
df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
df_results

Running ADE-DQN-CMA-ES-EGT on Rastrigin...


,Function,Optimum,Best,Worst,Mean,Std,Execution Time (s)
0,Rastrigin,"[0.9949586264558866, -0.9949586381743692, 0.99...",7.959667,84.571116,38.0839,13.337491,9144.13264


## **Ackley**

In [ ]:
def evaluate_function(func, x):
    return func(x)

def ackley(x):
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e

test_functions = {"Ackley": ackley}
bounds = [(-32.768, 32.768)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

Running ADE-DQN-CMA-ES-EGT on Ackley...


## **EggHolder**

In [ ]:
def eggholder(x):
    return (-(x[1] + 47.0)
            * np.sin(np.sqrt(abs(x[0]/2.0 + (x[1] + 47.0))))
            - x[0] * np.sin(np.sqrt(abs(x[0] - (x[1] + 47.0)))))

test_functions = {"EggHolder": eggholder}
bounds = [(-512, 512)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Grienwank**

In [ ]:
def griewank(x):
    return 1 + np.sum(x**2) / 4000 - np.prod(np.cos(x / np.sqrt(np.arange(1, len(x) + 1))))

test_functions = {"Griewank": griewank}
bounds = [(-600, 600)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(func, bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Schwefei**

In [ ]:
def schwefel(x):
    """Schwefel function."""
    return 418.9829 * len(x) - sum(x_i * np.sin(np.sqrt(np.abs(x_i))) for x_i in x)

bounds = [(-500, 500)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history = AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Schwefei_2_22**

In [ ]:
def schwefel_2_22(x):
    return np.sum(np.abs(x)) + np.prod(np.abs(x))

test_functions = {
    "Schwefel_2_22": schwefel_2_22}

bounds = [(-10, 10)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Levy**

In [ ]:
def levy(x):
    w = 1 + (x - 1) / 4
    term1 = np.sin(np.pi * w[0])**2
    term2 = np.sum((w[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * w[:-1] + 1)**2))
    term3 = (w[-1] - 1)**2 * (1 + np.sin(2 * np.pi * w[-1])**2)
    return term1 + term2 + term3

test_functions = {"Levy": levy}
bounds = [(-10, 10)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Michalewicz**

In [ ]:
def michalewicz(x, m=10):
    return -sum(np.sin(x[i]) * (np.sin((i + 1) * x[i]**2 / np.pi))**(2 * m) for i in range(len(x)))

test_functions = {"Michalewicz": michalewicz}
bounds = [(-0, 3.14)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Zakharov**

In [ ]:
def zakharov(x):
    term1 = np.sum(x**2)
    term2 = np.sum(0.5 * (np.arange(1, len(x) + 1)) * x)**2
    term3 = np.sum(0.5 * (np.arange(1, len(x) + 1)) * x)**4
    return term1 + term2 + term3

test_functions = {"Zakharov": zakharov}
bounds = [(-5, 10)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Zdt1**

In [ ]:
# Multi-objective functions
def zdt1(x):
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / (len(x) - 1)
    f2 = g * (1 - np.sqrt(f1 / g))
    return f1, f2

test_functions = {"Zdt1": zdt1}

bounds = [(0, 1)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Zdt2**

In [ ]:
def zdt2(x):
    f1, g = zdt1(x)
    f2 = g * (1 - (f1 / g)**2)
    return f1, f2

test_functions = {
    "Zdt2": zdt2}

bounds = [(0, 1)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)

## **Dtlz2**

In [ ]:
def dtlz2(x):
    g = np.sum(x[1:])
    f1 = (1 + g) * np.cos(x[0] * np.pi / 2)
    f2 = (1 + g) * np.sin(x[0] * np.pi / 2)
    return f1, f2

test_functions = {
    "Dtlz2": dtlz2}

bounds = [(0, 1)] * 10

results = []
for name, func in test_functions.items():
    print(f"Running ADE-DQN-CMA-ES-EGT on {name}...")

    start_time = time.time()
    fitness_history =  AdaptiveEA(lambda x: evaluate_function(func, x), bounds, CONFIG["max_generations"], CONFIG["num_runs"])
    optimum, best, worst, mean, std = compute_metrics(fitness_history)
    end_time = time.time()

    execution_time = end_time - start_time
    results.append([name, optimum, np.min(best), np.max(worst), np.mean(mean), np.mean(std), execution_time])

df_results = pd.DataFrame(results, columns=["Function", "Optimum", "Best", "Worst", "Mean", "Std", "Execution Time (s)"])
print(df_results)